In [1]:
import torch
from torch import nn
from tqdm.auto import tqdm
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import make_grid
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
torch.manual_seed(0)
def show_tensor_images(image_tensor, num_images=25, size=(1, 28, 28)):
    image_unflat = image_tensor.detach().cpu().view(-1, *size)
    image_grid = make_grid(image_unflat[:num_images], nrow=5)
    plt.imshow(image_grid.permute(1, 2, 0).squeeze())
    plt.show()

The "sequential " helps to form the building blocks inside the generator_one class

In [2]:
def generator_one(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.BatchNorm1d(output_dim),
        nn.ReLU(inplace=True)
    )


"ASSERT" is used to define if the situation is true or false


In [3]:
def generator_one_test(input_features,output_features,num_test=3):
  test_block=generator_one(input_features,output_features)
  assert len(test_block) == 3
  assert type(test_block[0]) ==nn.Linear
  assert type(test_block[1]) ==nn.BatchNorm1d
  assert type(test_block[2]) ==nn.ReLU

  test_input=torch.randn(num_test,input_features)
  test_output=test_block(test_input)
  assert test_output.shape==(num_test,output_features)
  return test_output



In [4]:
print(generator_one_test(512,1024))

tensor([[0.0000, 0.0000, 0.0000,  ..., 1.4062, 1.2586, 0.0000],
        [0.6393, 1.2542, 1.2928,  ..., 0.0000, 0.0000, 1.3822],
        [0.7728, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],
       grad_fn=<ReluBackward0>)


In [5]:
class Generator(nn.Module):
    def __init__(self, z_samples_noise_vector=10, img_dimension=784, hidden_layer_dimension=128):
        super(Generator, self).__init__()
        self.gen = nn.Sequential(
            generator_one(z_samples_noise_vector, hidden_layer_dimension),
            generator_one(hidden_layer_dimension, hidden_layer_dimension * 2),
            generator_one(hidden_layer_dimension * 2, hidden_layer_dimension * 4),
            generator_one(hidden_layer_dimension * 4, hidden_layer_dimension * 8),
            nn.Linear(hidden_layer_dimension * 8, img_dimension),
            nn.Sigmoid()
        )

    def forward(self, noise):
      return self.gen(noise)

    def get_gen(self):
        return self.gen

In [6]:
# Example usage
z_samples = torch.randn(5, 10)  # Random noise vector of 5 samples, each with 10 dimensions
generator = Generator(z_samples_noise_vector=10, img_dimension=784)
generated_images = generator(z_samples)
print(generated_images.shape)  # Should print: torch.Size([5, 784])


torch.Size([5, 784])


In [7]:
def test_generator(z_dim, im_dim, hidden_dim, num_test=10000):
    gen = Generator(z_dim, im_dim, hidden_dim).get_gen()
    assert len(gen) == 6, "Generator should have exactly 6 layers"
    test_input = torch.randn(num_test, z_dim)
    test_output = gen(test_input)
    assert tuple(test_output.shape) == (num_test, im_dim), f"Expected output shape ({num_test}, {im_dim}), but got {test_output.shape}"
    assert test_output.max() <= 1, "Output values should be in the range [0, 1] after sigmoid activation"
    assert test_output.min() >= 0, "Output values should be in the range [0, 1] after sigmoid activation"
    assert 0.05 < test_output.std() < 0.15, f"Expected std between 0.05 and 0.15, but got {test_output.std()}"

# Test cases with realistic values
test_generator(10, 784, 128)  # Standard z_dim, im_dim, hidden_dim for MNIST-style generator
test_generator(100, 784, 128)  # Test with a larger noise vector (z_dim)
test_generator(10, 784, 256)  # Test with larger hidden dimension

print("Success!")


Success!


In [8]:
def test_own_generator(z_samples_noise_vector, img_dimension, hidden_layer_dimension,num_test=3):
  gen_test=Generator(z_samples_noise_vector,img_dimension,hidden_layer_dimension)
  assert len(gen_test.get_gen())==6
  test_own_input=torch.randn(num_test,z_samples_noise_vector)
  test_own_output=gen_test.get_gen()(test_own_input)
  assert test_own_output.shape==(num_test,img_dimension)
  print("excellent ")

test_own_generator(15,17,128)

excellent 


**GENERATING NOISE**

In [9]:
def noise_generation(num_samples,z_samples_noise_vector,device='cpu'):
  return torch.randn(num_samples,z_samples_noise_vector,device=device)
  print("excellent")

noise_generation(2,5)

tensor([[-0.0029, -0.6079, -0.4270, -0.5158,  0.2514],
        [ 0.2405,  2.0934, -0.0604,  0.1784, -0.1186]])

In [10]:
def test_noise_generation(num_samples, z_samples_noise_vector):
    test_two = noise_generation(num_samples, z_samples_noise_vector)
    assert test_two.shape == (num_samples, z_samples_noise_vector), f"Expected shape ({num_samples}, {z_samples_noise_vector}), but got {test_two.shape}"
    print("success")
test_noise_generation(2, 3)


success


**BUILDING DISCRIMINATOR**

In [11]:
def get_discriminator_block(input_dim, output_dim):
    return nn.Sequential(
        nn.Linear(input_dim, output_dim),
        nn.LeakyReLU(negative_slope=0.2, inplace=True)
    )


In [12]:
# Verify the discriminator block function
def test_disc_block(in_features, out_features, num_test=10000):
    block = get_discriminator_block(in_features, out_features)
    assert len(block) == 2
    test_input = torch.randn(num_test, in_features)
    test_output = block(test_input)
    assert tuple(test_output.shape) == (num_test, out_features)
    assert -test_output.min() / test_output.max() > 0.1
    assert -test_output.min() / test_output.max() < 0.3
    assert test_output.std() > 0.3
    assert test_output.std() < 0.5

test_disc_block(25, 12)
test_disc_block(15, 28)
print("Success!")

Success!


In [13]:
class Discriminator(nn.Module):
    def __init__(self, im_dim=784, hidden_dim=128):
        super(Discriminator, self).__init__()
        self.disc = nn.Sequential(
            get_discriminator_block(im_dim, hidden_dim * 4),
            get_discriminator_block(hidden_dim * 4, hidden_dim * 2),
            get_discriminator_block(hidden_dim * 2, hidden_dim),
            nn.Linear(hidden_dim, 1)
        )

    def forward(self, image):
        return self.disc(image)

    def get_disc(self):
        return self.disc


In [14]:
# Verify the discriminator class
def test_discriminator(z_dim, hidden_dim, num_test=100):
  disc = Discriminator(z_dim, hidden_dim).get_disc()
  assert len(disc) == 4
  test_input = torch.randn(num_test, z_dim)
  test_output = disc(test_input)
  assert tuple(test_output.shape) == (num_test, 1)
  assert not isinstance(disc[-1], nn.Sequential)

test_discriminator(5, 10)
test_discriminator(20, 8)
print("Success!")

Success!


**LOADING DATASET**

BCEwithLogitsloss : turns the output of discriminator(logits) into the range between 0 and 1 and gets the difference between the predicted label and the true label

In [15]:
# Set your parameters
criterion = nn.BCEWithLogitsLoss()
n_epochs = 200
z_samples_noise_vector = 64
display_step = 500
batch_size = 128
lr = 0.00001

# Load MNIST dataset as tensors
dataloader = DataLoader(
    MNIST('.', download=True, transform=transforms.ToTensor()),
    batch_size=batch_size,
    shuffle=True)

### DO NOT EDIT ###
device = 'cpu'

Failed to download (trying next):
<urlopen error [Errno 111] Connection refused>



100%|██████████| 9.91M/9.91M [00:00<00:00, 16.9MB/s]


Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw

Failed to download (trying next):
<urlopen error [Errno 111] Connection refused>



100%|██████████| 28.9k/28.9k [00:00<00:00, 505kB/s]


Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw

Failed to download (trying next):
<urlopen error [Errno 111] Connection refused>



100%|██████████| 1.65M/1.65M [00:00<00:00, 4.53MB/s]


Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw

Failed to download (trying next):
<urlopen error [Errno 111] Connection refused>



100%|██████████| 4.54k/4.54k [00:00<00:00, 2.28MB/s]

Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw



In [16]:
gen=Generator(z_samples_noise_vector).to(device)
gen_opt=torch.optim.Adam(gen.parameters(),lr=lr)
disc=Discriminator().to(device)
disc_opt=torch.optim.Adam(disc.parameters(),lr=lr)
